### Import modules

In [17]:
import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions
import json
from openai import OpenAI

### Import tools (API client)

In [18]:
from tools import (
    get_top_selling_products,
    get_top_categories,
    get_sales_trends,
    get_revenue_by_category
)

### Initialize the LLM

In [19]:
model="gpt-4o"
llm = OpenAI()

### Initialize Vector DB

In [20]:
chroma_client = chromadb.PersistentClient(path="./data")
embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
collection = chroma_client.get_or_create_collection(name="products", embedding_function=embedding_function)

### Helper Functions

#### Map tools to prompt

In [21]:
def map_tools(prompt):
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_top_selling_products",
                "description": "Retrieve top-selling products for a specified period",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "start_date": {
                            "type": "string",
                            "description": "The start date for the period (YYYY-MM-DD)"
                        },
                        "end_date": {
                            "type": "string",
                            "description": "The end date for the period (YYYY-MM-DD)"
                        }
                    },
                    "required": ["start_date", "end_date"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_top_categories",
                "description": "Retrieve top-selling categories for a specified period",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "start_date": {
                            "type": "string",
                            "description": "The start date for the period (YYYY-MM-DD)"
                        },
                        "end_date": {
                            "type": "string",
                            "description": "The end date for the period (YYYY-MM-DD)"
                        }
                    },
                    "required": ["start_date", "end_date"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_sales_trends",
                "description": "Retrieve sales trends over a specified period",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "start_date": {
                            "type": "string",
                            "description": "The start date for the period (YYYY-MM-DD)"
                        },
                        "end_date": {
                            "type": "string",
                            "description": "The end date for the period (YYYY-MM-DD)"
                        }
                    },
                    "required": ["start_date", "end_date"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_revenue_by_category",
                "description": "Retrieve the revenue generated by each category over a specified period",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "start_date": {
                            "type": "string",
                            "description": "The start date for the period (YYYY-MM-DD)"
                        },
                        "end_date": {
                            "type": "string",
                            "description": "The end date for the period (YYYY-MM-DD)"
                        }
                    },
                    "required": ["start_date", "end_date"]
                }
            }
        }
    ]

    messages = [{"role": "user", "content": prompt}]
    response = llm.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    
    # Ensure response has valid tool_calls
    response_message = response.choices[0].message
    tool_calls = getattr(response_message, 'tool_calls', None)

    functions = []
    if tool_calls:
        for tool in tool_calls:
            function_name = tool.function.name
            arguments = json.loads(tool.function.arguments)
            functions.append({
                "function_name": function_name,
                "arguments": arguments
            })

    return functions

#### Execute tools locally (Invoke API)

In [22]:
import json

def execute_tools(functions):
    local_functions = {
        "get_top_selling_products": get_top_selling_products,
        "get_top_categories": get_top_categories,
        "get_sales_trends": get_sales_trends,
        "get_revenue_by_category": get_revenue_by_category
    }

    combined_results = []
    for detail in functions:
        function_name = detail["function_name"]
        arguments = detail["arguments"]
        function_result = local_functions[function_name](**arguments)
        if isinstance(function_result, list):
            combined_results.extend(function_result)
        else:
            combined_results.append(function_result)
    return combined_results

#### Retrieve from Context

In [23]:
def retriever(query):
    vector = embedding_function([query])
    results = collection.query(    
        query_embeddings=vector,
        n_results=5,
        include=["documents"]
    )
    res = " \n".join(str(item) for item in results['documents'][0])
    return res

In [24]:
retriever("connectivity options of Nimbus Book")

'With a wide range of connectivity options, including USB -C, HDMI, and a microSD card reader, the Nimbus Book 14 is versatile enough to connect to all your d evices and peripherals. \nRunning on the latest NimbusOS, the Nimbus Book 14 provides a smooth and intuitive user experience, with access to a wide range of productivity and entertainment apps. Whether you’re at home, in the office, or on the move, the Nimbus Book 14 is your reliable companion for all your computing needs. \nFeaturing a compact and lightweight design, the Nimbus Book 14 is perfect for those on the go. The 14 -inch Full HD IPS display delivers cri sp visuals and vibrant colors, making it ideal for creative work, presentations, and media consumption. \nPowered by the latest Intel i7 processor and 16GB of RAM, this laptop offers lightning -fast performance and seamless multitasking. The Nimbus Book 14 is equipped with a 512GB SSD, providing ample storage space for all your files, documents, and multimedia. \nThe Aur

#### Generate answer

In [25]:
def generate_response(prompt,context):
    input_text = (
        "Based on the below context, respond with an accurate answer. If you don't find the answer within the context, say I do not know. Don't repeat the question\n\n"
        f"{context}\n\n"
        f"{prompt}"
    )
    response = llm.chat.completions.create(
        model= model,
        messages=[
            {"role": "user", "content": input_text},
        ],
        max_tokens=150,
        temperature=0
    )

    return response.choices[0].message.content.strip()

#### Test helper functions

In [26]:
tools=map_tools("What was the top selling product in Q2 based on revenue?")

In [27]:
tools

[{'function_name': 'get_top_selling_products',
  'arguments': {'start_date': '2023-04-01', 'end_date': '2023-06-30'}}]

In [28]:
tool_output=execute_tools(tools)

In [29]:
#tool_output

### RAG Agent

In [30]:
def agent(prompt):
    tools = map_tools(prompt)
    
    if tools:    
        tool_output = execute_tools(tools)
        context = json.dumps(tool_output)       
    else:
        context = retriever(prompt)
        
    response = generate_response(prompt, context)
    return response

In [31]:
agent("What was the top selling category in Q2 based on revenue?")

'The top selling category in Q2 based on revenue was Electronics.'

In [32]:
agent("What is the CPU of Nimbus Book?")

'The CPU of the Nimbus Book 14 is the latest Intel i7 processor.'